In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import FastText
from gensim.models import KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
import gensim.downloader as api
from nltk.tokenize import word_tokenize
import nltk
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from math import sqrt
from sklearn.svm import SVC
import time
import tracemalloc
import xgboost as xgb

# Load the dataset
df = pd.read_csv('GrammarandProductReviews[modified].csv')


# Split the data into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(df['review'], df['positive_review'], test_size=0.2, random_state=42)

# Load the FastText and Glove pre-trained models
ft_model = api.load('fasttext-wiki-news-subwords-300')
glove_model = api.load("glove-wiki-gigaword-300")

# Define a function to extract hybrid features
def hybrid_features(text):
    ft_feature = np.mean([ft_model.get_vector(word) for word in text if word.strip()], axis=0)
    glove_feature = np.mean([glove_model.get_vector(word) for word in text if word.strip()], axis=0)
    return np.concatenate((ft_feature, glove_feature))


# Extract features for train and test sets
train_features = np.array([hybrid_features(text) for text in train_data])
test_features = np.array([hybrid_features(text) for text in test_data])

LogReg_clf = LogisticRegression(max_iter=1000)
SVC_clf = SVC(probability=True, kernel='rbf')
NV=MultinomialNB()
DTree_clf = DecisionTreeClassifier()
rf = RandomForestClassifier(random_state=1200, criterion='entropy', n_estimators=200)
ada = AdaBoostClassifier(n_estimators=200, learning_rate=1.0, algorithm='SAMME')
#ada = AdaBoostClassifier()
ETree = ExtraTreesClassifier(random_state=42)

#Voting Ensemble Method
voting_clf = VotingClassifier(estimators=[('svm', SVC_clf), ('AdaBoost', ada), ('RF',rf )], voting='soft')
voting_clf.fit(train_features, train_labels)
preds = voting_clf.predict(test_features)

acc = accuracy_score(test_labels, preds)
rms= sqrt(mean_squared_error(test_labels, preds))


print("Accuracy For Voting Ensemble SVM+AadBoost+RF is: " + str(acc))
print("RMSE Error is: " + str(rms))


# Train a BaggingClassifier using Logistic Regression as the base estimator
clf = BaggingClassifier(base_estimator=LogReg_clf, n_estimators=100, random_state=42)
clf.fit(train_features, train_labels)

# Make predictions on the test set
predictions = clf.predict(test_features)

# Evaluate the model using accuracy
acc = accuracy_score(test_labels, predictions)
rms= sqrt(mean_squared_error(test_labels, predictions))
print('Accuracy for Bagged LR:', acc)
print("RMSE Error is: " + str(rms))


# Train a BaggingClassifier using Logistic Regression as the base estimator
clf = BaggingClassifier(base_estimator=rf, n_estimators=100, random_state=42)
clf.fit(train_features, train_labels)

# Make predictions on the test set
predictions = clf.predict(test_features)

# Evaluate the model using accuracy
acc = accuracy_score(test_labels, predictions)
rms= sqrt(mean_squared_error(test_labels, predictions))
print('Accuracy for Bagged RF:', acc)
print("RMSE Error is: " + str(rms))

# Train a BaggingClassifier using DTree as the base estimator
clf = BaggingClassifier(base_estimator=DTree_clf, n_estimators=100, random_state=42)
clf.fit(train_features, train_labels)

# Make predictions on the test set
predictions = clf.predict(test_features)

# Evaluate the model using accuracy
acc = accuracy_score(test_labels, predictions)
rms= sqrt(mean_squared_error(test_labels, predictions))
print('Accuracy for Bagged DT:', acc)
print("RMSE Error is: " + str(rms))

clf = xgb.XGBClassifier(n_estimators=70, learning_rate=0.9)
clf.fit(train_features, train_labels)

# Make predictions on the test set
predictions = clf.predict(test_features)

# Evaluate the model using accuracy
acc = accuracy_score(test_labels, predictions)
rms= sqrt(mean_squared_error(test_labels, predictions))
print('Accuracy for Xboost:', acc)
print("RMSE Error is: " + str(rms))

